In [1]:
import requests as req
import lxml.html
import json
import re

In [2]:
def get_song_tags(artist = "", track = ""):
    tmp_artist = artist.lower().replace(' ', '+')
    tmp_track = track.lower().replace(' ', '+')
    url = f'https://www.last.fm/music/{tmp_artist}/_/{tmp_track}'
    html_from_last_fm = req.get(url).text
    html = lxml.html.fromstring(html_from_last_fm)
    track_dict = {'artist': artist, 'track': track, 'tags':[]}
    for li in html.xpath("//ul[@class='tags-list tags-list--global']/li"):
        track_dict['tags'].append(li.text_content())
    return track_dict

In [5]:
tags = []
tags.append(get_song_tags(artist='Elthon John', track='goodbye yellow brick road'))
tags.append(get_song_tags(artist='Григорий Лепс', track='рюмка водки'))
print(tags)

[{'artist': 'Elthon John', 'track': 'goodbye yellow brick road', 'tags': ['70s', 'classic rock', 'pop', 'elton john', 'rock']}, {'artist': 'Григорий Лепс', 'track': 'рюмка водки', 'tags': ['russian', 'rock', 'russian rock', 'chanson', 'male vocalists']}]
